In [22]:
import pandas as pd
import numpy as np
import torch
from transformers import pipeline

In [2]:
books = pd.read_csv('books_with_categories.csv')

In [3]:
# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
classifier = pipeline("text-classification",
                      model="j-hartmann/emotion-english-distilroberta-base",
                      top_k=None,
                      device=device)
classifier("I love this!")

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

C:\Users\Claudiu\Projects\semantic search\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Claudiu\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


[[{'label': 'joy', 'score': 0.9771687984466553},
  {'label': 'surprise', 'score': 0.008528673090040684},
  {'label': 'neutral', 'score': 0.005764589179307222},
  {'label': 'anger', 'score': 0.004419779404997826},
  {'label': 'sadness', 'score': 0.002092391485348344},
  {'label': 'disgust', 'score': 0.0016119909705594182},
  {'label': 'fear', 'score': 0.00041385178337804973}]]

In [9]:
books['description'][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [10]:
classifier(books['description'][0])

[[{'label': 'fear', 'score': 0.6548418998718262},
  {'label': 'neutral', 'score': 0.16985182464122772},
  {'label': 'sadness', 'score': 0.11640854179859161},
  {'label': 'surprise', 'score': 0.020700592547655106},
  {'label': 'disgust', 'score': 0.01910073682665825},
  {'label': 'joy', 'score': 0.015161239542067051},
  {'label': 'anger', 'score': 0.00393515033647418}]]

In [17]:
sentences = books['description'][0].split(".")
predictions = classifier(sentences)

In [19]:
print(sentences[4])
predictions[4]

 He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption


[{'label': 'sadness', 'score': 0.9671575427055359},
 {'label': 'neutral', 'score': 0.015104170888662338},
 {'label': 'disgust', 'score': 0.006480592768639326},
 {'label': 'fear', 'score': 0.005393994972109795},
 {'label': 'surprise', 'score': 0.0022869433742016554},
 {'label': 'anger', 'score': 0.0018428886542096734},
 {'label': 'joy', 'score': 0.001733880490064621}]

In [20]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

In [21]:
def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}


In [23]:
for i in range(10):
    isbn.append(books["isbn10"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

In [24]:
emotion_scores

{'anger': [np.float64(0.06413363665342331),
  np.float64(0.6126185059547424),
  np.float64(0.06413363665342331),
  np.float64(0.3514834940433502),
  np.float64(0.0814124047756195),
  np.float64(0.23222540318965912),
  np.float64(0.5381839871406555),
  np.float64(0.06413363665342331),
  np.float64(0.30066895484924316),
  np.float64(0.06413363665342331)],
 'disgust': [np.float64(0.27359241247177124),
  np.float64(0.3482835590839386),
  np.float64(0.10400672256946564),
  np.float64(0.15072275698184967),
  np.float64(0.18449535965919495),
  np.float64(0.7271743416786194),
  np.float64(0.1558549404144287),
  np.float64(0.10400672256946564),
  np.float64(0.27948135137557983),
  np.float64(0.17792679369449615)],
 'fear': [np.float64(0.928168535232544),
  np.float64(0.942527711391449),
  np.float64(0.9723208546638489),
  np.float64(0.36070650815963745),
  np.float64(0.0950433686375618),
  np.float64(0.05136279761791229),
  np.float64(0.7474278807640076),
  np.float64(0.40449780225753784),
  np

In [25]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn10"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [08:03<00:00, 10.76it/s]


In [29]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn10"] = isbn

In [30]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,neutral,isbn10
0,0.064134,0.273592,0.928169,0.932798,0.646217,0.967158,0.729603,0002005883
1,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545,0002261987
2,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765,0006178731
3,0.351483,0.150723,0.360707,0.251881,0.732685,0.111690,0.078765,0006280897
4,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765,0006280935
...,...,...,...,...,...,...,...,...
5192,0.148208,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656,8172235224
5193,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765,8173031010
5194,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625,817992162X
5195,0.064134,0.104007,0.459268,0.759456,0.951104,0.368112,0.078765,8185300534


In [31]:
books = pd.merge(books, emotions_df, on = "isbn10")

In [32]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise,neutral
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,0002005883 A NOVEL THAT READERS and critics ha...,Fiction,0.064134,0.273592,0.928169,0.932798,0.646217,0.967158,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,0002261987 A new 'Christie for Christmas' -- a...,Fiction,0.612619,0.348284,0.942528,0.704422,0.887939,0.111690,0.252545
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"0006178731 A memorable, mesmerizing heroine Je...",Fiction,0.064134,0.104007,0.972321,0.767237,0.549477,0.111690,0.078765
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,0006280897 Lewis' work on the nature of love d...,Nonfiction,0.351483,0.150723,0.360707,0.251881,0.732685,0.111690,0.078765
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"0006280935 ""In The Problem of Pain, C.S. Lewis...",Nonfiction,0.081412,0.184495,0.095043,0.040564,0.884390,0.475881,0.078765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,8172235224 On A Train Journey Home To North In...,Fiction,0.148208,0.030643,0.919165,0.255171,0.853722,0.980877,0.030656
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,8173031010 This book tells the tale of a man w...,Nonfiction,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,"817992162X Wisdom to Create a Life of Passion,...",Fiction,0.009997,0.009929,0.339217,0.947779,0.375755,0.066685,0.057625
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,8185300534 This collection of the timeless tea...,Nonfiction,0.064134,0.104007,0.459268,0.759456,0.951104,0.368112,0.078765


In [33]:
books.to_csv("books_with_emotions.csv", index = False)